In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from main_script import download_images_for_points
from process_data import get_gvi_per_buffer
from osmnx_road_network import select_points_on_road_network, get_features_on_points, get_road_network_with_points, select_points_within_buffers
import geopandas as gpd
from datetime import timedelta
from time import time

In [2]:
file = "Test_multiple_home_locations.gpkg" # File with address points
layer = "Test_multiple_home_locations"
buffer_distance = 500 # buffer distance in meters
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf" # Access token for mapillary

In [3]:
# Read the file as a GeoDataFrame
gdf_points = gpd.read_file(file, layer=layer) # The crs must use meters

gdf_points['buffer'] = gdf_points['geometry'].buffer(buffer_distance)

In [4]:
# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_points.to_crs(epsg=4326, inplace=True)

# Reproject the 'buffer' column to EPSG 4326
gdf_points['buffer'] = gdf_points['buffer'].to_crs(epsg=4326)

In [5]:
road = get_road_network_with_points(gdf_points['buffer'])

# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
road.to_crs(epsg=4326, inplace=True)

In [6]:
road_points = select_points_on_road_network(road)

In [7]:
points = select_points_within_buffers(gdf_points, road_points)

In [8]:
# Get features for each point
features = get_features_on_points(points, access_token)

file_path = os.path.join("results", "points.gpkg")
features.to_file(file_path, driver="GPKG")

# Get the initial time
start_time = time()
    
gvi_per_point = download_images_for_points(features, access_token)
# Get the final time
end_time = time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Format the elapsed time as "hh:mm:ss"
formatted_time = str(timedelta(seconds=elapsed_time))

print(f"Running time: {formatted_time}")

Running time: 0:26:47.204114


In [29]:
def get_gvi_per_buffer(buffered_points, gvi_per_point):
    gvi_per_point = gvi_per_point[gvi_per_point['missing'] == False]
    joined = gpd.sjoin(gvi_per_point, buffered_points.set_geometry('buffer'), how='inner', predicate='within')

    # Group the points by buffer
    grouped = joined.groupby('index_right')

    # Calculate the average 'gvi' for each group
    avg_gvi = grouped['GVI'].mean().reset_index()

    # Merge with the buffered_points dataframe to get the buffer geometries
    result = avg_gvi.merge(buffered_points, left_on='index_right', right_index=True)

    return gpd.GeoDataFrame(result[['geometry', 'GVI']])

In [30]:
results = get_gvi_per_buffer(gdf_points, gvi_per_point)
results

,geometry,GVI
0,POINT (-2.17238 53.43243),0.154665
1,POINT (-2.21250 53.44085),0.274186
2,POINT (-2.17267 53.45455),0.127187


In [31]:
file_path = os.path.join("results", "GVI-points.gpkg")
gvi_per_point.to_file(file_path, driver="GPKG")

file_path = os.path.join("results", "GVI.gpkg")
results.to_file(file_path, driver="GPKG")